In [1]:
from util import *
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from mlxtend.classifier import StackingCVClassifier
from sklearn.multiclass import  OneVsOneClassifier,OneVsRestClassifier

In [2]:
train_all = load_train()
test_all = load_testA()
preprocess_basic_time(train_all)
preprocess_basic_wifi(train_all)
shop_info = load_shop_info()

In [82]:
mall_id = "m_2270"
train = train_all[train_all.mall_id == mall_id]
# label
y = train.shop_id.values
le = LabelEncoder().fit(y)
y = le.transform(y)
#split
_train_index, _valid_index = get_last_one_week_index(train)
valid = train.iloc[_valid_index]
train = train.iloc[_train_index]
#wifi info
df, (train_index, train_use_wifi, train_matrix), (test_index, test_use_wifi, test_matrix) = get_wifi_cache2(mall_id)
train_wifi_all_x = train_matrix[_train_index]
valid_wifi_all_x = train_matrix[_valid_index]
valid_y = y[_valid_index]
train_y = y[_train_index]
train_lonlats = train[["longitude","latitude"]].values
valid_lonlats = valid[["longitude","latitude"]].values
train_wh = train[["weekday","hour"]].values
valid_wh = valid[["weekday","hour"]].values
train_w = train[["weekday"]].values
valid_w = valid[["weekday"]].values
train_h = train[["hour"]].values
valid_h = valid[["hour"]].values

In [80]:
# rf all_wifi_x
rf = RandomForestClassifier(n_estimators=500,n_jobs=-1,random_state=2017)
_train_x1 = np.concatenate([train_wifi_all_x,train_lonlats,train_h],axis=1)
_valid_x1 = np.concatenate([valid_wifi_all_x,valid_lonlats,valid_h],axis=1)
rf.fit(train_wifi_all_x,train_y)
acc(rf.predict(valid_wifi_all_x),valid_y)

0.8782736997417927

In [38]:
# rf2 strong wifi
indexs = choose_string_wifi_index(-115,6,train_wifi_all_x)
rf2 = RandomForestClassifier(n_estimators=500,n_jobs=-1,random_state=2017)
rf2.fit(train_wifi_all_x[:,indexs],train_y)
acc(rf2.predict(valid_wifi_all_x[:,indexs]),valid_y)

0.8829174664107485

In [50]:
# rf + rf2
acc(valid_y,
    rf2.classes_.take(np.argmax(rf.predict_proba(valid_wifi_all_x) + 
                                rf2.predict_proba(valid_wifi_all_x[:,indexs]),
                                axis=1)))

0.8794625719769674

In [88]:
# rf3 string wifi + lonlat
indexs = choose_strong_wifi_index(-90,6,train_wifi_all_x)
_train_x3 = np.concatenate([train_wifi_all_x[:,indexs],train_lonlats,train_wh],axis=1)
_valid_x3 = np.concatenate([valid_wifi_all_x[:,indexs],valid_lonlats,valid_wh],axis=1)
rf3 = RandomForestClassifier(n_estimators=500,n_jobs=-1,class_weight="balanced")
rf3.fit(_train_x3,train_y)
acc(rf3.predict(_valid_x3),valid_y)

0.9494097807757167

In [60]:
# rf2 + rf3
acc(valid_y,
    rf3.classes_.take(np.argmax(rf2.predict_proba(valid_wifi_all_x[:,indexs]) + 
                                rf3.predict_proba(_valid_x3),
                                axis=1)))

0.8879078694817658

In [64]:
# rf4 strong wifi lonlat hour
indexs = choose_string_wifi_index(-115,6,train_wifi_all_x)
_train_x4 = np.concatenate([train_wifi_all_x[:,indexs],train_lonlats,train_h],axis=1)
_valid_x4 = np.concatenate([valid_wifi_all_x[:,indexs],valid_lonlats,valid_h],axis=1)
rf4 = RandomForestClassifier(n_estimators=500,n_jobs=-1,random_state=2017)
rf4.fit(_train_x4,train_y)
acc(rf4.predict(_valid_x4),valid_y)

0.8871401151631478

In [66]:
# rf2 + rf3 + rf4
acc(valid_y,
    rf3.classes_.take(np.argmax(rf2.predict_proba(valid_wifi_all_x[:,indexs]) + 
                                rf3.predict_proba(_valid_x3) +
                                rf4.predict_proba(_valid_x4),
                                axis=1)))

0.8871401151631478

In [35]:
def expansion(trainx, trainy, cv=2):
    # 对样本少的进行复制扩充
    bin = np.bincount(trainy)
    labels = np.unique(trainy)
    l = np.asarray(range(np.max(labels) + 1))[bin < cv]
    l = np.intersect1d(l, labels)
    for _l in l:
        n = (trainy == _l).sum()
        n = int(np.ceil(float(cv) / n - 1))
        trainx = np.concatenate([trainx, np.tile(trainx[trainy == _l], (n, 1))], axis=0)
        trainy = np.concatenate([trainy, np.tile(trainy[trainy == _l], (n,))], axis=0)
    return trainx, trainy

In [56]:
# ovr 
ovr = OneVsRestClassifier(estimator=RandomForestClassifier(n_estimators=188,n_jobs=-1,class_weight="balanced"),n_jobs=-1)
ovr.fit(_train_x3, train_y)
acc(ovr.predict(_valid_x3),valid_y)

0.8905950095969289

In [71]:
# ovo
ovo = OneVsOneClassifier(estimator=RandomForestClassifier(n_estimators=50,n_jobs=-1),n_jobs=-1)
ovo.fit(_train_x3, train_y)
acc(ovo.predict(_valid_x3),valid_y)

0.8840690978886756

In [84]:
#stack
cv = 3
_x,_y = expansion(_train_x3,train_y,cv)
stack = StackingCVClassifier([RandomForestClassifier(n_estimators=500,n_jobs=-1,class_weight="balanced"),
                              OneVsRestClassifier(estimator=RandomForestClassifier(n_estimators=188,n_jobs=-1, class_weight="balanced"),n_jobs=-1),
                              ],
                             RandomForestClassifier(n_estimators=300,n_jobs=-1,class_weight="balanced"),
                             use_probas=True,
                             verbose=1,
                             cv = cv) 
stack.fit(_x,_y)
p = stack.predict(_valid_x3)
acc(p, valid_y)

Fitting 2 classifiers...
Fitting classifier1: randomforestclassifier (1/2)
Training and fitting fold 1 of 3...
Training and fitting fold 2 of 3...
Training and fitting fold 3 of 3...
Fitting classifier2: onevsrestclassifier (2/2)
Training and fitting fold 1 of 3...
Training and fitting fold 2 of 3...
Training and fitting fold 3 of 3...


0.9498313659359191